## Задача: "Я удалил таблицу, но файлы остались"
1. Создайте DataFrame и сохраните его как External Table с именем sales_history в папку data/sales_history.

In [1]:
from pyspark.sql import SparkSession
import os

# Включаем поддержку Hive (хотя в локальном докере это часто эмуляция через Derby)
spark = SparkSession.builder \
    .appName("Lab_05_03_Catalog") \
    .enableHiveSupport() \
    .getOrCreate()

# 1. Создаем df
df = spark.createDataFrame([(1, "Item A"), (2, "Item B")], ["id", "name"])
external_path = "/home/jovyan/workspace/data/sales_history_new"
df.write.mode("overwrite").parquet(external_path)

spark.sql(f"""
    CREATE TABLE sales_history_new (
        id INT,
        name STRING
    )
    USING PARQUET
    LOCATION '{external_path}'
""")

DataFrame[]

In [2]:
# где лежит
spark.sql("DESCRIBE EXTENDED sales_history_new").filter("col_name = 'Location'").show(truncate=False)
# Проверим тип
spark.sql("DESCRIBE EXTENDED sales_history_new").filter("col_name = 'Type'").show()

+--------+--------------------------------------------------+-------+
|col_name|data_type                                         |comment|
+--------+--------------------------------------------------+-------+
|Location|file:/home/jovyan/workspace/data/sales_history_new|       |
+--------+--------------------------------------------------+-------+

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|    Type| EXTERNAL|       |
+--------+---------+-------+



### 2. Выполните SQL: DROP TABLE sales_history.

In [3]:
spark.sql("DROP TABLE sales_history_new")

DataFrame[]

### 3. Проверьте через терминал (или os.listdir), остались ли файлы в папке data/sales_history. (Должны остаться).

In [4]:
!ls -la /home/jovyan/workspace/data/

total 28
drwxr-xr-x  7 jovyan users 4096 Jan 15 13:15 .
drwxrwxr-x 10 jovyan  1000 4096 Dec 18 11:53 ..
drwxr-xr-x  2 jovyan users 4096 Dec 16 14:24 example.json
drwxr-xr-x  2 jovyan users 4096 Jan 15 12:46 external_items
drwxr-xr-x  5 jovyan users 4096 Dec 16 12:24 processed_sales
drwxr-xr-x  2 jovyan users 4096 Dec 16 12:22 raw_sales
drwxr-xr-x  2 jovyan users 4096 Jan 15 13:15 sales_history_new


### 4. Восстановите доступ к данным. Создайте таблицу заново, указав тот же LOCATION.

In [5]:
spark.sql(f"""
    CREATE TABLE sales_history_new (
        id INT,
        name STRING
    )
    USING PARQUET
    LOCATION '{external_path}'
""")

DataFrame[]

### 5. Убедитесь, что SELECT снова работает.

In [6]:
spark.sql("SELECT * FROM sales_history_new")

DataFrame[id: int, name: string]